<h3> Importing Necessary Libraries and Modules </h3>
In this section, we'll import the necessary libraries and modules required for our data processing tasks.

In [2]:
import numpy as np
import pandas as pd
import datetime as dt
from src.data_handling import load_csv_data, closest_date, save_to_csv

<h3> Loading Data </h3> 

We'll load the data for different titles into a list of DataFrames. Each title corresponds to a CSV file that we'll read and preprocess.


In [3]:
# Define the path and titles
PATH = "DATA/raw"
DF_TITLES = ["AAPL", "AMZN", "MSFT", "SPX", "VIX", "IRX"]

# Function to load and preprocess data
def load_data(title: str) -> pd.DataFrame:
    temp_df = pd.read_csv(f"{PATH}/{title}.csv")
    
    # Create new column names
    new_columns = ["Date"] + [f"{title}_{col}" for col in temp_df.columns if col != "Date"]
    
    # Check if the number of new column names matches the number of columns in the DataFrame
    if len(new_columns) != len(temp_df.columns):
        raise ValueError(f"Column name mismatch for {title}.csv")
    
    temp_df.columns = new_columns
    temp_df["Date"] = pd.DatetimeIndex(temp_df["Date"])
    return temp_df.set_index("Date")


# Load data into a list of DataFrames
df_list = [load_data(title) for title in DF_TITLES]

<h3> Data Preprocessing </h3>

In this section, we'll:
1. Filter the data based on the latest start date.
2. Handle exceptions for specific titles.
3. Concatenate the data into a single DataFrame.


In [4]:
# Find the latest start date
latest_start = max([df.index[0] for df in df_list])
earliest_end = min(([df.index[-1] for df in df_list]))
# Apply date filter
df_list = [df[df.index >= latest_start] for df in df_list]
df_list = [df[df.index <= earliest_end] for df in df_list]

# Handle exception for VIX
date_set = set(df_list[0].index)
df_list[4] = df_list[4][df_list[4].index.isin(date_set)]

# Concatenate to a single DataFrame and drop unnecessary columns
df = pd.concat(df_list, axis=1)
df = df.drop(["VIX_Volume", "IRX_Volume"], axis=1)

# Filtering weekends
df = df[[date.isoweekday() <= 5 for date in df.index]]

<h3> Handling Missing Dates</h3>

To ensure continuity in our time series data, we'll:
1. Identify missing dates.
2. Fill the missing dates with average values from neighboring dates.


In [5]:
# Identify all dates and the range
all_dates = set(df.index)
start_date, end_date = df.index[0], df.index[-1]
date_range = pd.date_range(start_date, end_date)

# Identify missing dates
missing_dates = date_range.difference(df.index)
missing_dates = missing_dates[missing_dates.to_series().apply(lambda x: x.isoweekday() <= 5)]

# Adding NaN's to missing dates
nan_rows = df[df.isna().any(axis=1)].index
missing_dates = missing_dates.append(nan_rows)

# Fill missing dates with average values
for date in missing_dates:
    closest_up, closest_down = closest_date(date, df)
    avg_data = df.loc[[closest_up, closest_down]].mean()
    df.loc[date] = avg_data

# Sort the DataFrame by date
df = df.sort_index()


<h3> Saving the Processed Data</h3>

Finally, we'll save the processed data to a CSV file for future use.


In [7]:
SAVE_PATH = "DATA/processed/df.csv"

save_to_csv(df, SAVE_PATH)

Data saved to DATA/processed/df.csv
